# Import Items

### Import Libraries

In [1]:
import glob
import os
import json
import xml.etree.ElementTree as ET
import requests
import urllib.request
import time
import configparser
import tqdm

### Get configuration file

In [2]:
config = configparser.ConfigParser()
config.sections()
config.read('../config/api.ini')
client_id = config.get('main', 'client_id')
client_secret = config.get('main', 'client_secret')
endpoint = config.get('main', 'endpoint')

### Connect to the Api

In [3]:
params = {
    'key_identity': client_id,
    'key_credential': client_secret
}

access_url = endpoint
print("Connected to:    ", access_url)

Connected to:     http://localhost:8080/api/


### Navigate the xml and converts it to json

In [4]:
dictionary = {}
fileName = ''

for file in glob.glob("../input/*"):
    root = ET.parse(file).getroot()
    fileName = str(os.path.basename(file))
    
    for group in root.findall('family/group'):
        groupName = group.attrib['name']
        item = group.findall('item')
        
        for value in item:
            fieldName = value.attrib['name']         
            matchingName = groupName +"_"+ fieldName
            fieldValue = value.text
            
            #fix getty links 
            if "getty" in str(fieldValue):
                a = fieldValue.find('subjectid=')
                replaceElement = fieldValue[a+len("subjectid=")]
                
                for i in range(0,7): # 7 is the lenght of the getty ID
                    replaceElement += fieldValue[a+len("subjectid=") + i]
                fieldValue = "http://vocab.getty.edu/page/tgn/" +  replaceElement
            
            cleanedText = str(fieldValue).replace("None", "").replace("\n", "").replace("\t", " ").replace('"', '').replace("'", "")
            
            # Creating/appending to a JSON structure
            if fileName not in dictionary:
                dictionary[fileName] = { matchingName :  [cleanedText] }
            else:
                if matchingName not in dictionary[fileName]:
                    dictionary[fileName][matchingName] = [cleanedText]
                else:
                    dictionary[fileName][matchingName].append(cleanedText)

### Define basic omeka items

In [6]:
omekaTemplate =  { "o:resource_template": {"@id": endpoint + "/resource_templates\/2","o:id": 2}}
omekaClass =  { "o:resource_class": {"@id": endpoint + "/resource_classes\/26","o:id": 26} }

### Convert Data Fields

In [9]:
for i in tqdm.tqdm(dictionary.keys()):
    
    # Converting entries
    objectExtent = { "dcterms:format": [ { "type": "literal", "property_id": 9, "@value": dictionary[fileName]['Objekt_Gem\u00e4lde Ma\u00dfe max. (H x B x T in cm)'][0] } ]}
    objectMaterial = { "ecrm:P45_consists_of": [ { "type": "literal", "property_id": 545, "@value": dictionary[fileName]['Objekt_Gem\u00e4lde Material'][0] } ]}
    identifier = { "dcterms:identifier": [ { "type": "literal", "property_id": 10, "@value": dictionary[i]['Objekt_Sammlungsobjekt-ID'][0] } ]}
    
    postFields = json.dumps({ **omekaClass, **omekaTemplate, **identifier})
    
    # Post Request
    url = access_url + 'items/'
    headers = {"Content-Type": "application/json"}
    #r = requests.post(url, data=postFields, params = params, headers=headers)
    time.sleep(.25)

  0%|          | 3/641 [00:00<02:59,  3.55it/s]


KeyboardInterrupt: 